In [ ]:
!pip install datasets

In [4]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset
from datasets import load_dataset_builder
from datasets import get_dataset_split_names


In [ ]:
ds_builder = load_dataset_builder("wikimedia/structured-wikipedia", "20240916.en")

In [ ]:
ds_builder.info.features

In [ ]:
get_dataset_split_names("wikimedia/structured-wikipedia", "20240916.en")

['train']

In [5]:
datasets = load_dataset(
    "wikimedia/structured-wikipedia", "20240916.en", split="train", streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/43.5k [00:00<?, ?B/s]

In [6]:
datasets

IterableDataset({
    features: ['name', 'identifier', 'abstract', 'version', 'event', 'url', 'date_created', 'date_modified', 'is_part_of', 'in_language', 'license', 'description', 'sections', 'infoboxes', 'main_entity', 'additional_entities', 'image'],
    num_shards: 1
})

In [26]:
# feature data types. Need to know for processing the data.
# need to iterate over more to get all, since empty features will return NoneType
i = 0
for entry in datasets:
  for key in entry.keys():
    print(f"{key} is {type(entry[key])}")
  print("\n\n")
  i += 1
  if i == 1: break

name is <class 'str'>
identifier is <class 'int'>
abstract is <class 'NoneType'>
version is <class 'dict'>
event is <class 'dict'>
url is <class 'str'>
date_created is <class 'datetime.datetime'>
date_modified is <class 'datetime.datetime'>
is_part_of is <class 'dict'>
in_language is <class 'dict'>
license is <class 'list'>
description is <class 'str'>
sections is <class 'list'>
infoboxes is <class 'NoneType'>
main_entity is <class 'NoneType'>
additional_entities is <class 'NoneType'>
image is <class 'NoneType'>





In [28]:
# Subsection types/names Examples (when looking at the links, the dataset seems to miss some subsections)
i = 0
for entry in datasets:
  print(entry['name'])
  print(entry['url'])
  print(type(entry['sections']))
  print(len(entry['sections']))
  for i in range(len(entry['sections'])):
    print(entry['sections'][i]['name'])
  print("===============")
  i += 1
  if i == 3: break

Not Again SU
https://en.wikipedia.org/wiki/Not_Again_SU
<class 'list'>
3
Abstract
Introduction:
Demands:


In [18]:
# Looking at examples of the sections feature, this is where the text is stored and also where the hyperlinks are stored.

i = 0
for entry in datasets:
  print(entry['sections'][0])
  i += 1
  if i == 5: break


{'type': 'section', 'name': 'Abstract', 'has_parts': None}
{'type': 'section', 'name': 'Abstract', 'has_parts': [{'type': 'list', 'value': None, 'links': None, 'name': None, 'has_parts': [{'type': 'list_item', 'value': 'REDIRECT List of Suits characters#Mike Ross', 'links': [{'url': 'https://en.wikipedia.org/wiki/List_of_Suits_characters#Mike_Ross', 'text': 'List of Suits characters#Mike Ross', 'images': None}], 'has_parts': None, 'name': None}]}]}
{'type': 'section', 'name': 'Abstract', 'has_parts': [{'type': 'paragraph', 'value': 'REDIRECT # Second Polish Republic', 'links': [{'url': 'https://en.wikipedia.org/wiki/Second_Polish_Republic', 'text': 'Second Polish Republic', 'images': None}], 'name': None, 'has_parts': None}]}
{'type': 'section', 'name': 'Abstract', 'has_parts': [{'type': 'list', 'value': None, 'links': None, 'name': None, 'has_parts': [{'type': 'list_item', 'value': 'REDIRECT Writing system#Directionality', 'links': [{'url': 'https://en.wikipedia.org/wiki/Writing_syste

In [ ]:
# Example of feature extraction.
# Trying to pull out number of links per entry, the urls of the links, the length of the text in characters, and the text itself.

dataset_urls = []
dataset_links = []
dataset_charlengths = []
dataset_texts = []

# we will look at i number of entried from the streaming dataset, the value for i is set at the end of the for loop
i = 0

for entry in datasets:
  # Only looks at first section, which is Abstract. Need to loop over full section list later
  # type(entry['sections']) is List
  links = 0
  urls =[]
  charlength = 0
  text = ""
  ### print(entry['name'])
  ### print(entry['url'])

  # check if 'sections' is empty or y
  if isinstance(entry['sections'], list):
    for key in entry['sections'][0].keys():
      subsection = entry['sections'][0][key]
      if key == 'has_parts' and isinstance(subsection, list):
        for sub_dict in subsection:
          for sub_key in sub_dict.keys():

            # Get text and character length from "value" if "type": "paragraph"
            if sub_dict[sub_key] == 'paragraph':
              charlength += len(sub_dict['value'])
              text += sub_dict['value']

            # Get link count and URLs from "links" if "links" contains a list (i.e. not NoneType)
            if sub_key == 'links':
              if isinstance(sub_dict[sub_key], list):
                links += len(sub_dict[sub_key])
                for link in sub_dict[sub_key]:
                  urls.append(link['url'])

            # If "has_parts" contains a list (i.e. not NoneType) then there is another layer that could contain more links
            if sub_key == 'has_parts' and isinstance(sub_dict[sub_key], list):
              subsubsection = sub_dict[sub_key]
              for subsub_dict in subsubsection:
                for subsub_key in subsub_dict.keys():
                  if subsub_key == 'links':
                    if isinstance(subsub_dict[subsub_key], list):
                      links += len(subsub_dict[subsub_key])
                      for link in subsub_dict[subsub_key]:
                        urls.append(link['url'])

                  # Check if there is another layer deeper, didn't find any after checking 10,000 entries
                  if subsub_key == 'has_parts' and isinstance(subsub_dict[subsub_key], list):
                    print("##there is even deeper to go")

  # append features to dataset-wide lists
  dataset_texts.append(text)
  dataset_charlengths.append(charlength)
  dataset_urls.append(urls)
  dataset_links.append(links)

  i += 1
  if i == 500: break

print(dataset_links)
print(dataset_urls)
print(dataset_charlengths)
# print(dataset_texts)

In [31]:
# This one looks over all sections in sections
# Example of feature extraction.
# Trying to pull out number of links per entry, the urls of the links, the length of the text in characters, and the text itself.

dataset_urls = []
dataset_links = []
dataset_charlengths = []
dataset_texts = []

# we will look at i number of entried from the streaming dataset, the value for i is set at the end of the for loop
i = 0

for entry in datasets:
  links = 0
  urls =[]
  charlength = 0
  text = ""
  ### print(entry['name'])
  ### print(entry['url'])

  # check if 'sections' is empty or y
  if isinstance(entry['sections'], list):
    for section in entry['sections']:
      for key in section.keys():
        # subsection = entry['sections'][0][key]
        if key == 'has_parts' and isinstance(section[key], list):
          for sub_dict in section[key]:
            for sub_key in sub_dict.keys():

              # Get text and character length from "value" if "type": "paragraph"
              if sub_dict[sub_key] == 'paragraph':
                charlength += len(sub_dict['value'])
                text += sub_dict['value']

              # Get link count and URLs from "links" if "links" contains a list (i.e. not NoneType)
              if sub_key == 'links':
                if isinstance(sub_dict[sub_key], list):
                  links += len(sub_dict[sub_key])
                  for link in sub_dict[sub_key]:
                    urls.append(link['url'])

              # If "has_parts" contains a list (i.e. not NoneType) then there is another layer that could contain more links
              if sub_key == 'has_parts' and isinstance(sub_dict[sub_key], list):
                subsubsection = sub_dict[sub_key]
                for subsub_dict in subsubsection:
                  for subsub_key in subsub_dict.keys():
                    if subsub_key == 'links':
                      if isinstance(subsub_dict[subsub_key], list):
                        links += len(subsub_dict[subsub_key])
                        for link in subsub_dict[subsub_key]:
                          urls.append(link['url'])

                    # Check if there is another layer deeper --> found some, need to work this out
                    if subsub_key == 'has_parts' and isinstance(subsub_dict[subsub_key], list):
                      print("##there is even deeper to go")

  # append features to dataset-wide lists
  dataset_texts.append(text)
  dataset_charlengths.append(charlength)
  dataset_urls.append(urls)
  dataset_links.append(links)

  i += 1
  if i == 10000: break

print(dataset_links)
# print(dataset_urls)
print(dataset_charlengths)
# print(dataset_texts)

Streaming output truncated to the last 5000 lines.
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deeper to go
##there is even deepe

In [33]:
print(max(dataset_charlengths))
print(max(dataset_links))

38197
2070


In [25]:
# Confirming the above works

i = 0
for entry in datasets:
  print(f"Entry {i+1}: ")
  print(entry['url'])
  if isinstance(entry['sections'], list):
    for section in entry['sections']:
      print(section)
      # for key in section.keys():
      #   print(key)
  print("\n")
  i += 1
  if i == 5: break

Entry 1: 
https://en.wikipedia.org/wiki/Not_Again_SU
{'type': 'section', 'name': 'Abstract', 'has_parts': None}
{'type': 'section', 'name': 'Introduction:', 'has_parts': [{'type': 'paragraph', 'value': 'The hashtag and student led organization #NotAgainSU began circulating after several racist incidents occurred on the campus of Syracuse University, during the academic course of 2019-2021. Initial reports of racist paraphernalia occurred early November in the dorm buildings of first- and second-year undergraduate students. After nearly a month of daily reports, Syracuse students organized a sit-in where they occupied, and created an elaborate list of 19 demands for Chancellor Kent Syverud to sign. After several protests, some taking place at his persona residence, the list was signed with few revisions on November 21st.', 'links': None, 'name': None, 'has_parts': None}]}
{'type': 'section', 'name': 'Demands:', 'has_parts': [{'type': 'paragraph', 'value': "The #NotAgainSU protestors mad